> **Bank "churn" dataset**
<br>` 'Exited' is our classification target `
<br>` 1 - went elsewhere (nonzero is True) `
<br>` 0 - remains as a customer `

**Import the main libraries**

In [ ]:
import pandas as pd
import numpy as np

from time import time

_import the local library_

In [ ]:
# add parent folder path where lib folder is
import sys
if ".." not in sys.path:import sys; sys.path.insert(0, '..') 

In [ ]:
from mylib import show_labels_dist, show_metrics, bias_var_metrics

**Import the Dataset**

In [ ]:
## file path: windows style
data = pd.read_csv('..\\datasets\\churn_modelling.csv')

## file path: unix style
#data = pd.read_csv('../datasets/churn_modelling.csv')

# shape method gives the dimensions of the dataset
print('Dataset dimensions: {} rows, {} columns'.format(
    data.shape[0], data.shape[1]))

In [ ]:
data.info()

In [ ]:
data.head(2)

***
**Data Preparation and EDA** (unique to this dataset)
* _Check for missing values_
* _Quick visual check of unique values_
* _Split the classification feature out of the dataset_
* _Check column names of categorical attributes ( for get_dummies() )_
* _Check column names of numeric attributes ( for Scaling )_

**Check for missing values**

In [ ]:
cnt=0
print('Missing Values - ')
for col in data.columns:
    nnul = pd.notnull(data[col]) 
    if (len(nnul)!=len(data)):
        cnt=cnt+1
        print('\t',col,':',(len(data)-len(nnul)),'null values')
print('Total',cnt,'features with null values')

# address missing values here

**Quick visual check of unique values, deal with unique identifiers**

In [ ]:
# Identify columns with only one value 
# or with number of unique values == number of rows
n_eq_one = []
n_eq_all = []

print('Unique value count (',data.shape[0],'Rows in the dataset )')
for col in data.columns:
    lc = len(data[col].unique())
    print(col, ' ::> ', lc)
    if lc == 1:
        n_eq_one.append(data[col].name)
    if lc == data.shape[0]:
        n_eq_all.append(data[col].name)

In [ ]:
# Drop columns with only one value
if len(n_eq_one) > 0:
    print('Dropping single-valued features')
    print(n_eq_one)
    data.drop(n_eq_one, axis=1, inplace=True)

# Drop or bin columns with number of unique values == number of rows
if len(n_eq_all) > 0:
    print('Dropping unique identifiers')
    print(n_eq_all)
    data.drop(n_eq_all, axis=1, inplace=True)

# continue with featue selection / feature engineering

In [ ]:
# Remove one more column that will not help predict the outcome
data.drop(['Surname'], axis=1, inplace=True)
data.head(2)

* let's "bin" Age, Balance, and EstimatedSalary<br> 
_we could try to guess at the categories,_<br> 
_but let's use a function from our local library_

In [ ]:
from mylib import autobin

In [ ]:
# before we start, let's shorten one feature name))
data.rename(columns={'EstimatedSalary': 'Salary'}, inplace = True)

In [ ]:
for feat in ['Age', 'Balance', 'Salary']:
    print(feat, '- minValue: ', data[feat].min(), 
          '  maxValue: ', data[feat].max())

In [ ]:
for feat in ['Age', 'Balance', 'Salary']:
    cuts, cats = autobin(data, feat)

    newcol = feat + 'Range'
    data[newcol] = pd.cut(data[feat], bins=cuts, labels=cats) 
    
    print('Unique value count:',feat,len(data[feat].unique()),
          '  ',newcol,len(data[newcol].unique())-1,'\n')

In [ ]:
data.info()

In [ ]:
data.head(2)

In [ ]:
data['BalanceRange'].value_counts(sort=False)

In [ ]:
# let's drop the detail and keep the categories
#   Using inPlace makes permanent changes to the dataframe in memory 
#   otherwise drop() will not affect the dataset we are working on
for feat in ['Age', 'Balance', 'Salary']:
    data.drop([feat], axis=1, inplace=True)

In [ ]:
data.info()

**<br>Classification target feature**
<br>"the Right Answers", or more formally "the desired outcome"
<br>Must be in a separate dataset for classification ,,,

In [ ]:
# this is a binary classification dataset
twoclass = True

In [ ]:
## 'Exited' is our classification target 
## 1 (nonzero is True) - went elsewhere, zero - remains as a customer
print(data['Exited'].value_counts())

In [ ]:
## Text labels look better in the confusion matrix

## a 'lambda' function is always simple, used only once
#data.Exited = data.Exited.apply(lambda x: 'Gone' if x==1 else 'Here')

## an alternative to a 'lambda' that has the same effect
data['Exited'] = ['Gone' if x==1 else 'Here' for x in data['Exited']]

## Let's change the name to 'Status' too - 'rename' is like 'drop'
## setting the parameter 'inplace' to True changes the original DataFrame 
## if not set, a new DataFrame is returned
data.rename(columns={'Exited': 'Status'}, inplace = True)

data['Status'].value_counts()

In [ ]:
data.info()

* Split the classification feature out of the dataset 

In [ ]:
## Feature being predicted ("the Right Answer")
labels_col = 'Status'
y = data[labels_col]

## Features used for prediction 
X = data.copy()
X.drop(labels_col, axis=1, inplace=True)

**<br>Check column names of categorical attributes**
<br>Features with text values (categorical attributes) need to be normalised
<br>by changing them to numeric types that the algorithms find easier to work with

In [ ]:
categori = X.select_dtypes(include=['object','category']).columns
print(categori.to_list())

In [ ]:
# check the distribution of the feature values 
for col in categori:
    print('Distribution of categories in', col)
    print(X[col].value_counts())
    print()

* 'one hot' encoding transforms a single column of text values into 
multiple columns of discrete values: 
it creates a new column for each unique value and puts
(one) in the column for which it is true and (zero) in the others

In [ ]:
Country = pd.get_dummies(X.Geography)
Country.head()

In [ ]:
X = pd.concat([X, Country], axis=1)
X.drop('Geography', axis=1, inplace=True)
X.info()

In [ ]:
# the automatic way adds the original feature name
X = pd.get_dummies(X)

In [ ]:
X.info()

**<br>Check column names of numeric attributes**
<br>Features with numeric values need to be normalised by changing the values to
small numbers in a specific range (scaling). _Note that scaling comes_ after _the test//train split!_

In [ ]:
numeri = X.select_dtypes(include=['float64','int64']).columns
print(numeri.to_list())

***

**<br>Create Test // Train Datasets**
> Split X and y datasets into Train and Test subsets,<br>keeping relative proportions of each class (stratify)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test =train_test_split(X, y,
                                                   test_size=0.2,
                                                   random_state=50,
                                                   stratify=y)
# train_test_split does random selection, 
#      so we should reset the dataframe indexes
X_train.reset_index(inplace=True, drop=True)
X_test.reset_index(inplace=True, drop=True)
y_train.reset_index(inplace=True, drop=True)
y_test.reset_index(inplace=True, drop=True)

***
Next are standard steps for all datasets: _scaling, classifiers, results_

**Scaling** comes _after_ test // train split

In [ ]:
# data before normalization
X_train.head()

In [ ]:
# scaling the Numeric columns 
# StandardScaler range: -1 to 1, MinMaxScaler range: zero to 1

# from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

# sklearn docs say 
#   "Don't cheat - fit only on training data, then transform both"
#   fit() expects 2D array: reshape(-1, 1) for single col or (1, -1) single row

for i in numeri:
    arr = np.array(X_train[i])
    scale = MinMaxScaler().fit(arr.reshape(-1, 1))
    X_train[i] = scale.transform(arr.reshape(len(arr),1))

    arr = np.array(X_test[i])
    X_test[i] = scale.transform(arr.reshape(len(arr),1))
    

In [ ]:
# data after normalization
X_train.head()

**<br>Classifier Selection**

In [ ]:
# prepare list
models = []

##  --  Linear  --  ## 
#from sklearn.linear_model import LogisticRegression 
#models.append (("LogReg",LogisticRegression())) 
#from sklearn.linear_model import SGDClassifier 
#models.append (("StocGradDes",SGDClassifier())) 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis 
models.append(("LinearDA", LinearDiscriminantAnalysis())) 
#from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis 
#models.append(("QuadraticDA", QuadraticDiscriminantAnalysis())) 

##  --  Support Vector  --  ## 
#from sklearn.svm import SVC 
#models.append(("SupportVectorClf", SVC())) 
#from sklearn.svm import LinearSVC 
#models.append(("LinearSVC", LinearSVC())) 
#from sklearn.linear_model import RidgeClassifier
#models.append (("RidgeClf",RidgeClassifier())) 

##  --  Non-linear  --  ## 
#from sklearn.tree import DecisionTreeClassifier 
#models.append (("DecisionTree",DecisionTreeClassifier())) 
#from sklearn.naive_bayes import GaussianNB 
#models.append (("GaussianNB",GaussianNB())) 
#from sklearn.neighbors import KNeighborsClassifier 
#models.append(("K-NNeighbors", KNeighborsClassifier())) 

##  --  Ensemble: bagging  --  ## 
#from sklearn.ensemble import RandomForestClassifier 
#models.append(("RandomForest", RandomForestClassifier())) 
##  --  Ensemble: boosting  --  ## 
#from sklearn.ensemble import AdaBoostClassifier 
#models.append(("AdaBoost", AdaBoostClassifier())) 
#from sklearn.ensemble import GradientBoostingClassifier 
#models.append(("GradientBoost", GradientBoostingClassifier())) 

##  --  NeuralNet (simplest)  --  ## 
#from sklearn.linear_model import Perceptron 
#models.append (("SingleLayerPtron",Perceptron())) 
#from sklearn.neural_network import MLPClassifier 
#models.append(("MultiLayerPtron", MLPClassifier()))

print(models)

**<br>Target Label Distributions** (standard block)

In [ ]:
# from our local library
show_labels_dist(X_train,X_test,y_train,y_test)

**<br>Fit and Predict** (standard block)

In [ ]:
# evaluate each model in turn
results = []

print('macro average: unweighted mean per label')
print('weighted average: support-weighted mean per label')
print('MCC: correlation between prediction and ground truth')
print('     (+1 perfect, 0 random prediction, -1 inverse)\n')

for name, clf in models:
    trs = time()
    print('Confusion Matrix:', name)
    
    clf.fit(X_train, y_train)
    ygx = clf.predict(X_test)
    results.append((name, ygx))
    
    tre = time() - trs
    print ("Run Time {} seconds".format(round(tre,2)) + '\n')
    
# Easy way to ensure that the confusion matrix rows and columns
#   are labeled exactly as the classifier has coded the classes
#   [[note the _ at the end of clf.classes_ ]]

    show_metrics(y_test, ygx, clf.classes_)   # from our local library
    print('\nParameters: ', clf.get_params(), '\n\n')

**Bias - Variance Decomposition** (standard block)

In [ ]:
# from our local library
# reduce (cross-validation) folds for faster results
folds = 20
for name, clf in models:
    print('Bias // Variance Decomposition:', name)
    bias_var_metrics(X_train,X_test,y_train,y_test,clf,folds)

***

***

In [ ]:
# save our original datasets before we test the reduced ones
XtrainOriginal = X_train
XtestOriginal = X_test

**<br>_These examples only work with one classifier_<br>**

***
**Feature Importance Permutation**
<br>This can be used with any classifier or regressor to estimate feature importance.
<br>
It returns two arrays: the first array (here: `imp_vals`) contains the actual 
   importance values we are interested in. The second array is assigned to ` _ ` because we are not using it.
When `num_rounds` > 1 the permutation is repeated multiple times 
   with different random seeds, and the first array holds 
   the average of the importance values,
   with all individual values from these runs in the second array. 

In [ ]:
# for graphs
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# works best with numeric values for the target feature
from sklearn.preprocessing import LabelEncoder
## Feature being predicted ("the Right Answer")
ynum = LabelEncoder().fit_transform(y_train)

In [ ]:
# note: for roc_auc_score
# we need to predict the probabilities, 
#     y_prob = clf.predict_proba(X_test)
# instead of predicting the class (like above)
#     y_pred = clf.predict(X_test)
#
# multiclass (ovr) works fine with
#     roc_auc_score(y_test, y_prob)
# but for binary classification
# output from model.predict_proba() is a matrix with 2 columns, one for each class
# to calculate roc, we need to provide the probability of the positive class:
#     roc_auc_score(y_test, y_prob[:,1])

In [ ]:
# note: for feature_importance_permutation
# metric MUST be "r2" for regression 
#        or "accuracy" for classification 
#        or a custom function with signature func(y_true, y_pred)

from sklearn.metrics import roc_auc_score

def ovr_roc_auc(y_true, y_pred):
    if twoclass:
        return roc_auc_score(y_true, y_pred[:,1], 
                             average='macro')
    else:
        return roc_auc_score(y_true, y_pred,
                             multi_class='ovr',
                             average='macro')

In [ ]:
# Create a list of the feature names for the graph
cols = list(X_train.columns)

In [ ]:
from mlxtend.evaluate import feature_importance_permutation
clf = models[0][1]
clf.fit(X_train.values, ynum)

imp_vals, _ = feature_importance_permutation(
    predict_method=clf.predict_proba,      ## see note
    X=X_train.values,                      ## cannot use dataframe
    y=ynum,                                ## numeric labels
    metric=ovr_roc_auc,                    ## custom scorer
    num_rounds=1, seed=1)

#print(imp_vals)
#zz = sorted(zip(imp_vals,cols),reverse=True)
#list(zz)

In [ ]:
midf = pd.DataFrame({'Name': X_train.columns, 'Score': imp_vals})
#midf.head()

nf = 8     # number of features
# extract the top nf
mihi = midf.sort_values('Score', ascending=False).head(nf)
#mihi

nf = 8     # number of features
# extract the low nf
milo = midf.sort_values('Score', ascending=True).head(nf)
#milo

# merge
hilo = pd.concat([mihi, milo])

In [ ]:
hilo

In [ ]:
# quick sns.barplot
ptitle = models[0][0]
ptitle += ": feature importance via permutation"
sns.barplot(x = "Score", y = "Name", data = hilo).set(title=ptitle)
plt.show()

In [ ]:
# create new dataframes with just the most important columns
rfeXtrain = X_train.filter(list(hilo['Name']))
rfeXtest = X_test.filter(list(hilo['Name']))

rfeXtrain.info()

In [ ]:
# substitute the datasets
X_train = rfeXtrain
X_test = rfeXtest

**<br>Fit and Predict** (standard block)

**Bias - Variance Decomposition** (standard block)

***
**SequentialFeatureSelector**
<br>Sequential feature selection algorithms are a family of greedy search algorithms that are used to reduce an initial d-dimensional feature space to a k-dimensional feature subspace where k < d. 
<br><br>
The motivation behind feature selection algorithms is to automatically select a subset of features most relevant to the problem. The goal of feature selection is two-fold: We want to improve the computational efficiency and reduce the model's generalization error by removing irrelevant features or noise.
<br><br>
In a nutshell, SFAs remove or add one feature at a time based on the classifier performance until a feature subset of the desired size k is reached. The 'floating' algorithms have an additional exclusion or inclusion step to remove features once they were included (or excluded) so that a larger number of feature subset combinations can be sampled.
<br><br>
http://rasbt.github.io/mlxtend/user_guide/feature_selection/SequentialFeatureSelector/ 
<br>has tutorial videos, diagrams<br>
http://rasbt.github.io/mlxtend/api_subpackages/mlxtend.feature_selection/#sequentialfeatureselector <br>documents the parameters

In [ ]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs

In [ ]:
# For SFS we can group one-hot encoded features as a single feature
# feature_groups : the features within a group are always selected together, never split
# HOWEVER
# the `feature_group` list must contain ALL features and
# there should be no common feature betweeen any two distinct groups of features provided 

# from above:
#      put the names into a python list - for pandas.get_dummies()
#      categori = combined_df.select_dtypes(include=['object']).columns
# ((there is probably a more elegant way to do this, but never mind ...))

# create the groups of features
g = []
for c in categori.tolist():
    n = []
    r = X_train.columns.str.startswith(c)
    for x in range(len(r)):
        if r[x]:
            n.append(x)
    g.append(n)
#print(g)

# get the others and add each one as an individual list
a = []
w = X_train.columns.tolist()
for x in range(len(w)):
    a.append(x)
r=a
for x in range(len(g)):
        q=list(set(r).difference(g[x]))
        r=q
for x in range(len(r)):
    z=[]
    z.append(x)
    g.append(z)
    
print(g)

***
***patch*** - *only for the demo (churn) notebook!!*

In [ ]:
#  [Geography was not done automatically by pandas]
g[0] = [6, 7, 8]
g=g[:-3]
print(g)

***

In [ ]:
go_fwd = False
 
sfs = SFS(models[0][1], 
          k_features='parsimonious', 
          forward=go_fwd, 
          floating=True,
          cv=2,      # CVfolds
          feature_groups=g,
          scoring='roc_auc_ovr',
          verbose=0, 
          n_jobs= -1)

sfs = sfs.fit(X_train, y_train)

In [ ]:
#import matplotlib.pyplot as plt
fig1 = plot_sfs(sfs.get_metric_dict(), kind='std_dev')

if go_fwd:
    plt.title('Sequential [Forward] Selection (w. StdDev)')
else:
    plt.title('Sequential [Backward] Selection (w. StdDev)')
    
#plt.ylim([0.8, 1])
plt.grid()
plt.show()

In [ ]:
sfs.k_feature_idx_

In [ ]:
sfs.k_feature_names_

In [ ]:
# uncomment the line below to see a dataframe of all of the details
#pd.DataFrame.from_dict(sfs.get_metric_dict()).T

In [ ]:
# create new dataframes with just those columns
sfsXtrain = X_train.filter(sfs.k_feature_names_)
sfsXtest = X_test.filter(sfs.k_feature_names_)

sfsXtrain.info()

In [ ]:
# substitute the datasets
X_train = sfsXtrain
X_test = sfsXtest

**<br>Fit and Predict** (standard block)

**Bias - Variance Decomposition** (standard block)

***
***